In [4]:
# Basic
from datetime import datetime, timedelta
import time

# Data Analysis Specific
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.parser import parse as date_parser
from geopy.geocoders import Nominatim

# Marchine Learning Specific
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV

# IPython magic
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
DATA_PATH = './data/'
DETROIT_LAT = (42.252, 42.452)
DETROIT_LNG = (-83.295, -82.895)
SIZE_RATIO = 1000
DETROIT_WIDTH = 200
DETROIT_HEIGHT = 400
print DETROIT_WIDTH, DETROIT_HEIGHT, float(DETROIT_WIDTH)/DETROIT_HEIGHT

200 400 0.5


In [3]:
raw_violation = pd.read_csv(DATA_PATH + 'detroit-blight-violations.csv', low_memory=False)
raw_311 = pd.read_csv(DATA_PATH + 'detroit-311.csv', low_memory=False)
raw_crime = pd.read_csv(DATA_PATH + 'detroit-crime.csv', low_memory=False)
raw_permit = pd.read_csv(DATA_PATH + 'detroit-demolition-permits.tsv.csv', low_memory=False)

In [132]:
def my_date_parser(dt):
    try:
        return date_parser(dt)
    except:
        return None

# Clean again permit

In [117]:
clean_permit = pd.read_csv('clean/permit.csv',index_col=0)

In [118]:
clean_permit['addr'] = raw_permit['SITE_ADDRESS']
clean_permit['turst'] = True
miss_permit_idx = clean_permit[['lat','lng']].isnull()[clean_permit.isnull()].dropna().index
clean_permit['turst'].ix[miss_permit_idx] = False

In [119]:
clean_permit.to_csv('clean/permit.csv')

In [120]:
clean_permit = pd.read_csv('clean/permit.csv',index_col=0)

In [191]:
def clean_again(df):
    assert df['lat'].isnull().sum() == df['lng'].isnull().sum(), 'input: lat(%d) and lng(%d) must be equal' % (df['lat'].isnull().sum(), df['lng'].isnull().sum())
    to_clean = df[df['turst']==False].copy()
    print 'Remain:', (to_clean['turst'] == False).sum()
    tursted = df[df['turst']==True].copy()
    geolocator = Nominatim()
    for idx in to_clean.index:
        addr = to_clean.loc[idx,'addr'] + ', Detroit'
        try:
            loc = geolocator.geocode(addr) 
            if loc is None:
                print 'Cannot parse %s' % addr
            else:
                lat, lng = float(loc.latitude), float(loc.longitude)
                if lat is not None and lng is not None:
                    to_clean.loc[idx,'lat'] = lat
                    to_clean.loc[idx,'lng'] = lng
            to_clean.loc[idx,'turst'] = True
        except Exception as e:
            print 'Exception: %s, maybe service time out.' % e
            break
    print 'Remain:', (to_clean['turst'] == False).sum()
    res = pd.concat([tursted, to_clean]).sort_index()
    assert res['lat'].isnull().sum() == res['lng'].isnull().sum(), 'output: lat(%d) and lng(%d) must be equal' % (res['lat'].isnull().sum(), res['lng'].isnull().sum())
    return res

In [194]:
clean_permit = clean_again(clean_permit)

Remain: 202
Exception: Service timed out, maybe service time out.
Remain: 202


In [195]:
clean_permit.to_csv('clean/permit.csv')

# Clean again violation

In [146]:
def violation_lat_lng():
    lat_lng = raw_violation['ViolationAddress'].map(lambda x: x.split('\n')[-1][1:-1])
    lat = lat_lng.map(lambda x: float(x.split(',')[0]))
    lng = lat_lng.map(lambda x: float(x.split(',')[1]))
    addr = pd.DataFrame()
    addr['lat'] = lat
    addr['lng'] = lng
    addr = addr.apply(lambda x: None if 42.331<=x.lat<42.332 and -83.048<=x.lng<-83.047 else x, axis = 1)
    addr['turst'] = True
    miss_idx = addr[['lat','lng']].isnull()[addr.isnull()].dropna().index
    addr['turst'].ix[miss_idx] = False
    addr['addr'] = raw_violation['ViolationAddress'].map(lambda x: x.split('\n')[0])
    addr['date'] = raw_violation['TicketIssuedDT'].map(lambda x: my_date_parser(x))

    return addr

In [147]:
clean_violation = violation_lat_lng()

In [148]:
clean_violation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307804 entries, 0 to 307803
Data columns (total 5 columns):
lat      286685 non-null float64
lng      286685 non-null float64
turst    307804 non-null bool
addr     307804 non-null object
date     268950 non-null object
dtypes: bool(1), float64(2), object(2)
memory usage: 9.7+ MB


In [149]:
clean_violation.to_csv('clean/violation.csv')

In [150]:
clean_violation = pd.read_csv('clean/violation.csv',index_col=0)

In [157]:
clean_violation = clean_again(clean_violation)

Remain: 21048
Time out, Service timed out
Remain: 21048


In [158]:
clean_violation.to_csv('clean/violation.csv')

In [166]:
tmp = raw_violation['JudgmentAmt'].map(lambda x: float(x[1:]) if str(x)[0] == '$' else float(x))

In [184]:
tmp.ix[tmp.isnull()] = 140.

In [187]:
clean_violation['JudgeAmt'] = tmp

In [196]:
clean_violation.to_csv('clean/violation.csv')

In [197]:
clean_violation = clean_again(clean_violation)

Remain: 21033
Cannot parse 657 MULLETT, Detroit
Cannot parse 312 WEST END, Detroit
Cannot parse 15326 G A, Detroit
Cannot parse 260 SCHWEITZER PL, Detroit
Cannot parse 260 SCHWEITZER PL, Detroit
Cannot parse 260 SCHWEITZER PL, Detroit
Cannot parse 260 SCHWEITZER PL, Detroit
Exception: Service timed out, maybe service time out.
Remain: 20919


In [198]:
clean_violation.to_csv('clean/violation.csv')

# clean 311 call

In [290]:
def three11_lat_lng():
    lat = raw_311['lat'].map(lambda x: float(x))
    lng = raw_311['lng'].map(lambda x: float(x))
    addr = pd.DataFrame()
    addr['lat'] = lat
    addr['lng'] = lng
    type_convert = {'Graffiti Abatement (internal use only, public issue)': 16, 'Customer Service (internal use only, private issue)': 14, 'Abandoned Vehicle': 4, 'DPW - Debris Removal': 17, 'Curbside Solid Waste Issue': 20, 'Trash Issue - Bulk waste deposited more than 24 hours before designated time': 19, 'Trash Issue - Improper placement of refuse container between collections/left at curbside': 18, 'Running Water in a Home or Building': 10, 'Clogged Drain': 0, 'Residential Snow Removal Issue': 15, 'Illegal Dumping / Illegal Dump Sites': 9, 'Water Main Break': 3, 'Traffic Sign Issue': 8, 'Test (internal use only, public issue)': 12, 'Manhole Cover Issue': 2, 'DPW - Other environmental': 22, 'Traffic Signal Issue': 6, 'Tree Issue': 1, 'Detroit Land Bank Referral': 21, 'Graffiti': 13, 'Street Light Pole Down': 11, 'Potholes': 7, 'Fire Hydrant Issue': 5}
    addr['Category'] = raw_311['issue_type'].map(lambda key: type_convert[key])
    addr['date'] = raw_311['ticket_last_updated_date_time'].map(lambda x: my_date_parser(x))
    return addr

In [291]:
clean_311 = three11_lat_lng()

In [292]:
clean_311.to_csv('clean/311.csv')

In [293]:
clean_311.Category.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22], dtype=int64)

# Clean crime

In [280]:
def crime_lat_lng():
    lat = raw_crime['LAT'].map(lambda x: float(x))
    lng = raw_crime['LON'].map(lambda x: float(x))
    addr = pd.DataFrame()
    addr['lat'] = lat
    addr['lng'] = lng
    category_convert = {'CONGRESS': 42, 'LIQUOR': 34, 'IMMIGRATION': 41, 'ENVIRONMENT': 28, 'FRAUD': 10, 'AGGRAVATED ASSAULT': 6, 'ROBBERY': 9, 'BURGLARY': 5, 'STOLEN PROPERTY': 25, 'OTHER BURGLARY': 24, 'HOMICIDE': 13, 'OBSCENITY': 36, 'MISCELLANEOUS ARREST': 48, 'TAX REVENUE': 38, 'JUSTIFIABLE HOMICIDE': 43, 'ANTITRUST': 32, 'ASSAULT': 0, 'FELONY DEATH FROM FLEEING VEHICLE': 47, 'WEAPONS OFFENSES': 3, 'KIDNAPING': 22, 'TRAFFIC VIOLATIONS-DRIVING ON SUSPENDED': 8, 'ELECTION LAWS': 45, 'GAMBLING': 40, 'VAGRANCY (OTHER)': 27, 'OBSTRUCTING THE POLICE': 19, 'DRUNKENNESS': 46, 'OBSTRUCTING JUDICIARY': 11, 'DANGEROUS DRUGS': 12, 'STOLEN VEHICLE': 2, 'REVOKED': 44, 'ARSON': 17, 'DAMAGE TO PROPERTY': 7, 'FAMILY OFFENSE': 16, 'LARCENY': 1, 'HEALTH-SAFETY': 26, 'OUIL DISPOSE OF VEHICLE TO AVOID FORFEITURE': 15, 'ESCAPE': 18, 'SOVEREIGNTY': 37, 'NEGLIGENT HOMICIDE': 39, 'EMBEZZLEMENT': 29, 'SOLICITATION': 14, 'FORGERY': 30, 'OUIL': 35, 'PUBLIC PEACE': 33, 'EXTORTION': 23, 'BRIBERY': 21, 'TRAFFIC VIOLATIONS-MOTORCYCLE VIOLATIONS': 4, 'MILITARY': 49, 'CONSPIRACY BY COMPUTER': 31, 'RUNAWAY': 20}
    addr['Category'] = raw_crime['CATEGORY'].map(lambda key: category_convert[key])
    addr['date'] = raw_crime['INCIDENTDATE'].map(lambda x: my_date_parser(x))
    return addr

In [281]:
clean_crime = crime_lat_lng()

In [282]:
clean_crime.to_csv('clean/crime.csv')

In [288]:
clean_crime.Category.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49], dtype=int64)